In [18]:
import random
import json
import os
import time
from schemdraw.parsing import logicparse


def generate_fixed_depth_expression(inputs, depth):
    """
    Generate a boolean expression with exactly the specified depth.
    Avoid consecutive 'not' operations and redundant nesting.
    """
    operators = ["and", "or", "xor"]  # Binary operators
    symbols = [chr(65 + i) for i in range(inputs)]  # A, B, C, D...

    def generate_expr(current_depth, allow_not=True):
        if current_depth == depth:
            # Base case: choose a symbol when depth is reached
            return random.choice(symbols)
        else:
            op = random.choice(operators + (["not"] if allow_not else []))
            if op == "not":
                # Avoid consecutive 'not' by disallowing 'not' in nested call
                if current_depth < depth - 1:
                    exp = f"not {generate_expr(current_depth, allow_not=False)}"
                else:
                    exp = f"not {generate_expr(current_depth + 1, allow_not=False)}"
                return exp
            else:
                # Binary operators increase depth
                left = generate_expr(current_depth + 1)
                right = generate_expr(current_depth + 1)
                # Ensure left and right are distinct
                while left == right:
                    right = generate_expr(current_depth + 1)
                # Add parentheses around left and right if they are not at the deepest layer
                if current_depth + 1 < depth:
                    left = f"({left})"
                    right = f"({right})"
                return f"{left} {op} {right}"

    return generate_expr(0)

def create_circuit_and_metadata(output_dir, num_circuits, inputs, depth):
    """
    Create circuits and save metadata and diagrams as individual JSON and SVG files.
    """
    os.makedirs(output_dir, exist_ok=True)

    for i in range(num_circuits):
        circuit_id = f"circuit_{i + 1}"
        print(f"Generating {circuit_id} ...")
        expression = generate_fixed_depth_expression(inputs, depth)
        # expression = "(not (((C or A) xor (C and C))))"
        print(expression)
        metadata_entry = {
            "circuit_id": circuit_id,
            "expression": expression,
            "inputs": [chr(65 + j) for j in range(inputs)],  # A, B, C, D...
            "depth": depth
        }

        # Save metadata to individual JSON file
        metadata_file = os.path.join(output_dir, f"{circuit_id}.json")
        with open(metadata_file, "w") as f:
            json.dump(metadata_entry, f, indent=4)

        # Create and save the circuit diagram
        try:
            print("Start parsing ...")
            tik = time.time()
            drawing = logicparse(expression, gateH=1.2)
            print(f"Parsing finished in {time.time() - tik} s")
            svg_file = os.path.join(output_dir, f"{circuit_id}.svg")
            drawing.save(svg_file)
        except Exception as e:
            print(f"Error generating circuit diagram for expression {expression}: {e}")

# Parameters
output_directory = "circuits_output"
number_of_circuits = 100  # Number of circuits to generate
number_of_inputs = 3     # Input variables (e.g., A, B, C, D)
fixed_depth = 3          # Fixed depth for all expressions

# Generate circuits and metadata
create_circuit_and_metadata(output_directory, number_of_circuits, number_of_inputs, fixed_depth)


Generating circuit_1 ...
not ((A and C) or (A xor C)) and ((B or C) or (C xor B))
Start parsing ...
Parsing finished in 0.23543596267700195 s
Generating circuit_2 ...
((C and B) xor (B xor A)) or ((not A) or (B and A))
Start parsing ...
Parsing finished in 0.16421294212341309 s
Generating circuit_3 ...
not ((A xor C) or (C or A)) xor ((B and A) xor (A and C))
Start parsing ...
Parsing finished in 0.23108410835266113 s
Generating circuit_4 ...
((A and B) or (C and B)) xor ((not A) or (A and B))
Start parsing ...
Parsing finished in 0.1570587158203125 s
Generating circuit_5 ...
((B or C) and (C or B)) or ((C xor A) xor (B and A))
Start parsing ...
Parsing finished in 0.15120410919189453 s
Generating circuit_6 ...
(not (C and A) xor (A xor C)) xor ((B xor A) xor (C or B))
Start parsing ...
Parsing finished in 0.2936398983001709 s
Generating circuit_7 ...
(not (not B) xor (C xor B)) xor ((B and C) or (C and B))
Start parsing ...
Parsing finished in 0.20365500450134277 s
Generating circuit_

KeyboardInterrupt: 

In [1]:
import cairosvg

def convert_svg_to_png(svg_path, png_path):
    # Convert SVG to PNG
    cairosvg.svg2png(url=svg_path, write_to=png_path)


OSError: no library called "cairo-2" was found
no library called "cairo" was found
no library called "libcairo-2" was found
cannot load library 'libcairo.so.2': dlopen(libcairo.so.2, 0x0002): tried: 'libcairo.so.2' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibcairo.so.2' (no such file), '/Users/will/miniconda3/envs/598/bin/../lib/libcairo.so.2' (no such file), '/usr/lib/libcairo.so.2' (no such file, not in dyld cache), 'libcairo.so.2' (no such file).  Additionally, ctypes.util.find_library() did not manage to locate a library called 'libcairo.so.2'
cannot load library 'libcairo.2.dylib': dlopen(libcairo.2.dylib, 0x0002): tried: 'libcairo.2.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibcairo.2.dylib' (no such file), '/Users/will/miniconda3/envs/598/bin/../lib/libcairo.2.dylib' (no such file), '/usr/lib/libcairo.2.dylib' (no such file, not in dyld cache), 'libcairo.2.dylib' (no such file).  Additionally, ctypes.util.find_library() did not manage to locate a li

In [2]:
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPM

# read svg -> write png
def convert_svg_to_png(svg_path, png_path):
    # Convert SVG to PNG
    renderPM.drawToFile(svg2rlg(svg_path), png_path, fmt='PNG')

In [3]:
convert_svg_to_png("/Users/will/libraries/VisualCoT/circuits_output/circuit_1.svg", "test.png")
# cairosvg.svg2png("/Users/will/libraries/VisualCoT/circuits_output/circuit_1.svg", "test.png")

Unable to find a suitable font for 'font-family:sans', weight:normal, style:normal
Unable to find a suitable font for 'font-family:sans', weight:normal, style:normal
Unable to find a suitable font for 'font-family:sans', weight:normal, style:normal
Unable to find a suitable font for 'font-family:sans', weight:normal, style:normal
Unable to find a suitable font for 'font-family:sans', weight:normal, style:normal
Unable to find a suitable font for 'font-family:sans', weight:normal, style:normal
Unable to find a suitable font for 'font-family:sans', weight:normal, style:normal
Unable to find a suitable font for 'font-family:sans', weight:normal, style:normal
Unable to find a suitable font for 'font-family:sans', weight:normal, style:normal
Unable to find a suitable font for 'font-family:sans', weight:normal, style:normal
Unable to find a suitable font for 'font-family:sans', weight:normal, style:normal
Unable to find a suitable font for 'font-family:sans', weight:normal, style:normal
Unab

RenderPMError: cannot import desired renderPM backend rlPyCairo
Seek advice at the users list see
https://pairlist2.pair.net/mailman/listinfo/reportlab-users

In [13]:
import cv2
import numpy as np

def detect_components_from_svg(svg_path, output_png_path="circuit_test3.png", output_with_boxes="output_with_components.png"):
    # Step 1: Convert SVG to PNG
    convert_svg_to_png(svg_path, output_png_path)
    
    # Step 2: Load the PNG with OpenCV
    img = cv2.imread(output_png_path)
    original = img.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Preprocessing: Apply GaussianBlur and Canny edge detection
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # List to store detected components
    components = []

    for contour in contours:
        # Get the bounding rectangle for each contour
        x, y, w, h = cv2.boundingRect(contour)
        
        # Filter out noise based on size (min width and height)
        if 80 < h < 120:  # Adjust thresholds as needed
            components.append({'x': x, 'y': y, 'width': w, 'height': h})
            
            # Draw bounding boxes on the original image for visualization
            cv2.rectangle(original, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Save the result with bounding boxes drawn
    cv2.imwrite(output_with_boxes, original)

    return components


In [14]:
# Input SVG path
svg_path = "circuits_output/circuit_1.svg"

# Detect components
components = detect_components_from_svg(svg_path)

# Print detected components
print("Detected Components:", components)


ValueError: invalid literal for int() with base 10: '238.208pt'

In [23]:
from sympy import symbols, sympify


# A, B = symbols('A B')

expr_ref = sympify("not (B and A) or (B or A)")
expr_output = sympify("(A or B)")



In [24]:
print(expr_ref.simplify())
print(expr_output.simplify())

B
A


In [6]:
from boolean import BooleanAlgebra

# 初始化布尔代数
algebra = BooleanAlgebra()

# 从字符串解析布尔表达式
expr_output = "not (B and A) or (B or A)"
expr_output = "A ^ B"
expr_ref = "(not (B and A)) and B or A"

expr_output = algebra.parse(expr_output).simplify()
expr_ref = algebra.parse(expr_ref).simplify()

# 化简表达式
print(expr_output)
print(expr_ref)
print(expr_output == expr_ref)

ParseError: Unknown token for token: "^" at position: 2